In [3]:

from backtesting import Backtest, Strategy
from backtesting.test import EURUSD, SMA 
import numpy as np
import pandas as pd

In [4]:

import sys
sys.path.append('../../')

In [5]:
from config.constants import *

In [6]:
SYMBOL

{'XAUUSD': {'H4': 'XAUUSD/XAUUSD_H4 - XAUUSD_H4'}}

In [18]:
XAUUSD = pd.read_csv(PATH_DATA + SYMBOL['XAUUSD']['H4'] +'.csv',index_col='Time',parse_dates=True)

In [8]:
XAUUSD = pd.read_csv(PATH_DATA + SYMBOL['XAUUSD']['H4'] + '.csv',index_col='Time')

In [20]:
XAUUSD.index.name = None

In [21]:
XAUUSD

,Open,High,Low,Close,Volume
2007-01-01 00:00:00,635.679,636.194,635.009,635.497,95
2007-01-01 04:00:00,635.336,636.220,635.254,635.699,97
2007-01-01 08:00:00,635.780,636.225,635.201,635.993,99
2007-01-01 12:00:00,636.053,636.189,635.117,636.006,90
2007-01-01 16:00:00,635.902,636.185,635.006,635.989,96
...,...,...,...,...,...
2023-03-10 00:00:00,1831.428,1834.765,1827.974,1829.085,240
2023-03-10 04:00:00,1829.085,1834.575,1827.675,1833.405,240
2023-03-10 08:00:00,1833.415,1837.245,1830.778,1834.865,240
2023-03-10 12:00:00,1834.885,1867.348,1831.384,1862.975,240


In [11]:
EURUSD


,Open,High,Low,Close,Volume
2017-04-19 09:00:00,1.07160,1.07220,1.07083,1.07219,1413
2017-04-19 10:00:00,1.07214,1.07296,1.07214,1.07260,1241
2017-04-19 11:00:00,1.07256,1.07299,1.07170,1.07192,1025
2017-04-19 12:00:00,1.07195,1.07280,1.07195,1.07202,1460
2017-04-19 13:00:00,1.07200,1.07230,1.07045,1.07050,1554
...,...,...,...,...,...
2018-02-07 11:00:00,1.23390,1.23548,1.23386,1.23501,2203
2018-02-07 12:00:00,1.23501,1.23508,1.23342,1.23422,2325
2018-02-07 13:00:00,1.23422,1.23459,1.23338,1.23372,2824
2018-02-07 14:00:00,1.23374,1.23452,1.23238,1.23426,4065


In [23]:
EURUSD.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5000 entries, 2017-04-19 09:00:00 to 2018-02-07 15:00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    5000 non-null   float64
 1   High    5000 non-null   float64
 2   Low     5000 non-null   float64
 3   Close   5000 non-null   float64
 4   Volume  5000 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 234.4 KB


In [22]:
XAUUSD.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 25976 entries, 2007-01-01 00:00:00 to 2023-03-10 16:00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    25976 non-null  float64
 1   High    25976 non-null  float64
 2   Low     25976 non-null  float64
 3   Close   25976 non-null  float64
 4   Volume  25976 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 1.2 MB


In [14]:
def ATR(df, n):
    df = df.copy()
    df['High-Low'] = abs(df['High'] - df['Low'])
    df['High-PrevClose'] = abs(df['High'] - df['Close'].shift(1))
    df['Low-PrevClose'] = abs(df['Low'] - df['Close'].shift(1))
    df['TR'] = df[['High-Low', 'High-PrevClose',
                   'Low-PrevClose']].max(axis=1, skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    df = df.drop(['High-Low', 'High-PrevClose', 'Low-PrevClose'], axis=1)
    return df

In [15]:
def calculate_position_size(account_size, risk_percentage, entry_price, stop_loss):
    # Calculate the dollar amount at risk per trade
    dollar_at_risk = account_size * risk_percentage / 100

    # Calculate the distance between entry price and stop-loss
    distance = entry_price - stop_loss

    # Calculate the position size
    position_size = dollar_at_risk / distance

    return position_size

In [16]:
class Momentum(Strategy) :
    def init(self):
        self.data.df['ATR'] = ATR(self.data.df, 20)['ATR']
        self.data.df['direction'] = np.where(self.data.Close > self.data.Open, 'bull', 'bear')
        self.data.df['dir_count'] = self.data.df.groupby((self.data.df['direction'] != self.data.df['direction'].shift(1)).cumsum()).cumcount()+1
    

    def next(self):
        current_dir_count = self.data.df.loc[self.data.df.index[-1], 'dir_count']
        current_direction = self.data.df.loc[self.data.df.index[-1], 'direction']
        current_ATR = self.data.df.loc[self.data.df.index[-1], 'ATR']
        if current_dir_count >= 5 and current_direction == 'bull' and not self.position.is_short and not self.position.is_long:
            tp = self.data.Close[-1] + current_ATR * 0.5
            sl = self.data.Open[-1] 
            limit = self.data.Open[-1]
            if tp > limit > sl:
                # Place your buy logic here
                self.buy(sl=sl, tp=tp, limit=limit,size=0.01)
        if current_dir_count >= 5 and current_direction == 'bear' and not self.position.is_short and not self.position.is_long:
            tp = self.data.Close[-1] - current_ATR * 0.5
            sl = self.data.Open[-1] 
            limit = self.data.Open[-1]
            limit = self.data.Close[-1]
            if tp < limit < sl:
                self.sell(sl=sl, tp=tp,limit=limit,size=0.01)  # Place your buy logic here

In [24]:
bt = Backtest(XAUUSD, Momentum, cash=1000, margin=1/2000, commission=0.0002)
start = bt.run()
start

C:\Users\Win10\AppData\Local\Temp\ipykernel_24536\3753811498.py:1: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(XAUUSD, Momentum, cash=1000, margin=1/2000, commission=0.0002)
c:\Users\Win10\anaconda3\lib\site-packages\backtesting\backtesting.py:911: UserWarning: (2007-01-26 12:00:00) A contingent SL/TP order would execute in the same bar its parent stop/limit order was turned into a trade. Since we can't assert the precise intra-candle price movement, the affected SL/TP order will instead be executed on the next (matching) price/bar, making the result (of this trade) somewhat dubious. See https://github.com/kernc/backtesting.py/issues/119
  warnings.warn(
c:\Users\Win10\anaconda3\lib\site-packages\backtesting\backtesting.py:911: UserWarning: (2007-02-27 20:00:00) A contingent SL/TP order would execute in the same

Start                     2007-01-01 00:00:00
End                       2023-03-10 16:00:00
Duration                   5912 days 16:00:00
Exposure Time [%]                    1.077918
Equity Final [$]                    52.649629
Equity Peak [$]                   1073.398945
Return [%]                         -94.735037
Buy & Hold Return [%]              193.329158
Return (Ann.) [%]                  -13.680722
Volatility (Ann.) [%]               22.919057
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -95.095055
Avg. Drawdown [%]                  -37.705836
Max. Drawdown Duration     5785 days 08:00:00
Avg. Drawdown Duration     1960 days 11:00:00
# Trades                                  113
Win Rate [%]                        38.053097
Best Trade [%]                       1.130709
Worst Trade [%]                      -2.06418
Avg. Trade [%]                    

In [25]:
bt.plot()

c:\Users\Win10\anaconda3\lib\site-packages\backtesting\_plotting.py:119: UserWarning: Data contains too many candlesticks to plot; downsampling to '1D'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
c:\Users\Win10\anaconda3\lib\site-packages\backtesting\_plotting.py:145: FutureWarning: Passing method to Int64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  new_bar_idx = new_index.get_loc(mean_time, method='nearest')
c:\Users\Win10\anaconda3\lib\site-packages\backtesting\_plotting.py:145: FutureWarning: Passing method to Int64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  new_bar_idx = new_index.get_loc(mean_time, method='nearest')
c:\Users\Win10\anaconda3\lib\site-packages\backtesting\_plotting.py:145: FutureWarning: Passing method to Int64Index.get_loc is deprecated and w

Row(id='1407', ...)